<a href="https://colab.research.google.com/github/Vaycold/statistics_seminar/blob/main/%EC%9E%84%EC%8B%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import os
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings 
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from google.colab import files


warnings.filterwarnings('ignore')

os.environ['KAGGLE_USERNAME'] = 'kimgodbang'
os.environ['KAGGLE_KEY'] = '1c0e1716b23829d4381dcbced37ba49b'

!rm *.*
!kaggle competitions download -c sentiment-analysis-of-covid-19-related-tweets

train_df = pd.read_csv('training.csv')
test_df = pd.read_csv('validation.csv')

  0% 0.00/254k [00:00<?, ?B/s]
100% 254k/254k [00:00<00:00, 37.7MB/s]
  0% 0.00/518k [00:00<?, ?B/s]
100% 518k/518k [00:00<00:00, 71.4MB/s]


In [25]:
def labelsplit(labels) :
    
    label = labels.replace(' ',',').split(',')
    for idx, lab in enumerate(label) :
        label[idx] = int(lab)
    return label

In [26]:
train_df['label']  = train_df['Labels'].apply(lambda x :  labelsplit(x))
target = ['0','1','2','3','4','5','6','7','8','9','10']

onehot = np.zeros((5000,11))
for i in range(5000) : # 0~5000 row
    for j in train_df.label[i] : # j = value   
        onehot[i,j]  = 1

onehot = pd.DataFrame(onehot, columns=target)

In [27]:
train_df = pd.concat(
    [train_df, onehot], axis=1
).drop(['ID','Labels', 'label'],axis=1)

In [28]:
x = train_df['Tweet']
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x)
sequences = tokenizer.texts_to_sequences(x)
word_index = tokenizer.word_index #word index

threshold = 2
total_cnt = len(word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합(vocabulary)에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

등장 빈도가 1번 이하인 희귀 단어의 수: 5572
단어 집합(vocabulary)에서 희귀 단어의 비율: 57.10186513629842
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 6.542440147005295


In [29]:
tokenizer = Tokenizer(num_words = total_cnt - rare_cnt + 1)
tokenizer.fit_on_texts(x)
sequences = tokenizer.texts_to_sequences(x)
word_index = tokenizer.word_index

In [30]:
vocab_size = len(word_index) + 1
print('단어 집합의 크기: {}'.format((vocab_size)))

단어 집합의 크기: 9759


In [31]:
n_of_train = int(len(sequences) * 0.8)
n_of_test = int(len(sequences) - n_of_train)
print('훈련 데이터의 개수 :',n_of_train)
print('테스트 데이터의 개수:',n_of_test)

훈련 데이터의 개수 : 4000
테스트 데이터의 개수: 1000


In [32]:
x_data = sequences
max_len = 29
data = pad_sequences(x_data, maxlen=max_len)
data.shape

(5000, 29)

In [33]:
y = train_df['0']

## Auto

In [282]:
def RNN_df() :
    df = pd.DataFrame()
    pdf = pd.DataFrame()
    test_seq = tokenizer.texts_to_sequences(test_df['Tweet'])
    test = pad_sequences(test_seq, maxlen=max_len)
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
    submission = test_df.drop(['Tweet'], axis=1)
    submission['Labels'] = 0

    for sentiment in target :
        y = train_df[sentiment]
        x_test = data[n_of_train : ]
        x_train = data[:n_of_train]
        y_test = np.array(y[n_of_train:])
        y_train = np.array(y[:n_of_train])
        model = Sequential()
        model.add(Embedding(vocab_size, 32)) # 임베딩 벡터의 차원은 32
        model.add(SimpleRNN(32)) # RNN 셀의 hidden_size는 32
        model.add(Dense(1, activation='sigmoid'))
        model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
        model.fit(x_train, y_train, epochs=4, batch_size=64, validation_split=0.2, verbose = 0,
                  callbacks = [es])
        res = model.predict(test)
        answer = pd.DataFrame(res, columns = [sentiment])
        pdf[sentiment] = answer[sentiment]
        df[sentiment] = answer[sentiment].apply(lambda x : 1 if x >= 0.5 else 0)
        print( sentiment ,  ' 완료')
    
    n_df = np.array(df)
    n_pdf = np.array(pdf)
    for row in range(2500) : # 행별로 돌아요
        if n_pdf[row].max() <= 0.5 :
            for idx, value in enumerate(n_pdf[row]) :
                if value == n_pdf[row].max() :
                    n_df[row][idx] = 1
    onehot_result = pd.DataFrame(n_df, columns = target)

    for row in range(2500) :
        one_row = []
        for cat in target :
            if int(onehot_result[row:row+1][cat].values) == 1 :
                one_row.append(str(cat))
        submission.Labels[row] = ' '.join(one_row)
    
    submission.to_csv('test.csv', index=False)

    files.download('test.csv')
    #return submission

In [283]:
RNN_df()

0  완료
1  완료
2  완료
3  완료
4  완료
5  완료
6  완료
7  완료
8  완료
9  완료
10  완료


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [223]:
onehot_

,0,1,2,3,4,5,6,7,8,9,10
0,1,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,1
3,1,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
2495,0,0,0,0,0,0,0,0,1,0,0
2496,1,0,0,0,0,0,0,0,0,0,0
2497,0,0,0,0,0,0,1,0,0,0,0
2498,0,0,0,0,0,0,0,0,0,0,1


In [150]:
for i in target :
    print(rnn_1[i].value_counts())

0    2260
1     240
Name: 0, dtype: int64
0    2483
1      17
Name: 1, dtype: int64
0    2499
1       1
Name: 2, dtype: int64
0    2460
1      40
Name: 3, dtype: int64
0    2228
1     272
Name: 4, dtype: int64
0    2263
1     237
Name: 5, dtype: int64
0    1604
1     896
Name: 6, dtype: int64
0    2453
1      47
Name: 7, dtype: int64
0    2311
1     189
Name: 8, dtype: int64
0    2371
1     129
Name: 9, dtype: int64
0    1307
1    1193
Name: 10, dtype: int64


In [230]:
rnn_1

,0,1,2,3,4,5,6,7,8,9,10
0,0,0,0,0,1,1,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
2495,0,0,0,0,0,0,0,0,0,0,0
2496,0,0,0,0,1,0,1,0,0,0,0
2497,0,0,0,0,0,0,1,0,0,0,0
2498,0,0,0,0,0,0,0,0,0,0,1


In [276]:
onehot_.columns

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'], dtype='object')

In [252]:
onehot_ = onehot_.astype(int)

In [268]:
type(onehot_['0'])

TypeError: ignored

In [269]:
type(rnn_1['0'])

pandas.core.series.Series

In [246]:
type(np.array(rnn_1)[0][0])

numpy.int64

In [232]:
submission

,ID,Labels
0,5001,0
1,5002,0
2,5003,0
3,5004,0
4,5005,0
...,...,...
2495,7496,0
2496,7497,0
2497,7498,0
2498,7499,0


In [277]:
submission = test_df.drop(['Tweet'], axis=1)
submission['Labels'] = 0

for row in range(2500) :
    one_row = []
    for cat in target :
        if int(onehot_[row:row+1][cat].values) == 1 :
            one_row.append(str(cat))
    submission.Labels[row] = ' '.join(one_row)

In [234]:
submission

,ID,Labels
0,5001,4 5 10
1,5002,
2,5003,10
3,5004,10
4,5005,4 10
...,...,...
2495,7496,
2496,7497,4 6
2497,7498,6
2498,7499,10


In [278]:
submission[submission['Labels']=='']

,ID,Labels


In [279]:
submission.head(5)

,ID,Labels
0,5001,0
1,5002,8
2,5003,10
3,5004,10
4,5005,10


In [204]:
rnn_1[rnn_1.sum(axis=1) == 0]

,0,1,2,3,4,5,6,7,8,9,10
1,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0,0
21,0,0,0,0,0,0,0,0,0,0,0
23,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2471,0,0,0,0,0,0,0,0,0,0,0
2474,0,0,0,0,0,0,0,0,0,0,0
2493,0,0,0,0,0,0,0,0,0,0,0
2495,0,0,0,0,0,0,0,0,0,0,0


In [156]:
rnn_p

,0,1,2,3,4,5,6,7,8,9,10
0,0.410295,0.022288,0.041154,0.236780,0.610259,0.549366,0.221764,0.018240,0.008997,0.080906,0.944885
1,0.091344,0.092740,0.084194,0.083643,0.036640,0.146146,0.114211,0.021444,0.414631,0.050704,0.167429
2,0.455104,0.086013,0.090872,0.085902,0.159103,0.082375,0.248575,0.055660,0.057558,0.168296,0.598394
3,0.101270,0.028003,0.017938,0.100716,0.036062,0.058501,0.204059,0.111721,0.020401,0.218555,0.943950
4,0.063119,0.037983,0.032834,0.096958,0.526858,0.054432,0.313862,0.020075,0.068195,0.075536,0.880046
...,...,...,...,...,...,...,...,...,...,...,...
2495,0.110606,0.148629,0.063709,0.168834,0.216944,0.063678,0.250917,0.102826,0.394547,0.169340,0.479171
2496,0.262739,0.070895,0.029620,0.351749,0.578544,0.335656,0.514132,0.029542,0.042643,0.194594,0.257083
2497,0.148868,0.088726,0.051726,0.122741,0.081545,0.106585,0.584198,0.015637,0.037286,0.217218,0.485389
2498,0.193843,0.067871,0.024294,0.093795,0.030045,0.067948,0.077407,0.063844,0.020444,0.017407,0.589972


In [194]:
plz2 = np.array(rnn_1)
plz2

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0]])

In [218]:
plz2[plz2.sum(axis=1) == 0]

array([], shape=(0, 11), dtype=int64)

In [198]:
plz2[0][10]

1

In [159]:
plz = np.array(rnn_p)
plz[1].max()
for i in range(2500) : # 행별로 돌아요
    if plz[i].max() < 0.5 :
        idx = 

0.41463134

In [207]:
plz[0].max()

0.9448852

In [208]:
for row in range(2500) : # 행별로 돌아요
    if plz[row].max() <= 0.5 :
        for idx, value in enumerate(plz[row]) :
            if value == plz[row].max() :
                plz2[row][idx] = 1


['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

In [219]:
plz22 = pd.DataFrame(plz2, columns=[target])
plz22

,0,1,2,3,4,5,6,7,8,9,10
0,0,0,0,0,1,1,0,0,0,0,1
1,0,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
2495,0,0,0,0,0,0,0,0,0,0,1
2496,0,0,0,0,1,0,1,0,0,0,0
2497,0,0,0,0,0,0,1,0,0,0,0
2498,0,0,0,0,0,0,0,0,0,0,1


In [193]:
for i, j in enumerate(plz[0]) :
    if j == plz[0].max() :
        print(i)

10


In [99]:
# 알고리즘
# label이 ''일 경우 solve함수 적용
def solve(x) :
    []

0     0.245757
1     0.023104
2     0.040764
3     0.045809
4     0.086896
5     0.033615
6     0.126789
7     0.022223
8     0.024177
9     0.046149
10    0.398423
Name: 3, dtype: float32

0.39842343

In [280]:
submission.to_csv('test.csv', index=False)
from google.colab import files
files.download('test.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [142]:
ct = 0
for i in range(2500) : 
    if plz[i].max() < 0.5 :
        ct += 1
ct

713

In [140]:
rnn_1[3:4]['10'] = 0
rnn_1

,0,1,2,3,4,5,6,7,8,9,10
0,0,0,0,0,0,1,0,0,0,0,0
1,1,0,0,0,0,0,0,0,1,0,0
2,1,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2495,0,0,0,0,0,0,0,0,1,0,0
2496,0,0,0,0,0,1,0,0,0,0,0
2497,0,0,0,0,0,0,1,0,0,0,0
2498,0,0,0,0,0,0,0,0,0,0,1


In [116]:
submission

,ID,Labels
0,5001,5
1,5002,0 8
2,5003,0 10
3,5004,
4,5005,0 4
...,...,...
2495,7496,8
2496,7497,5
2497,7498,6
2498,7499,10


In [110]:
submission.head(5)

,ID,Labels
0,5001,5
1,5002,0 8
2,5003,0 10
3,5004,
4,5005,0 4


In [143]:
submission[submission['Labels']=='']

,ID,Labels
3,5004,
7,5008,
15,5016,
34,5035,
35,5036,
...,...,...
2463,7464,
2465,7466,
2480,7481,
2492,7493,


0       False
1       False
2       False
3        True
4       False
        ...  
2495    False
2496    False
2497    False
2498    False
2499     True
Name: Labels, Length: 2500, dtype: bool